---
<big><big><big><big><big><big>Metody uczenia maszynowego</big></big></big></big></big></big>

---

---
<big><big><big><big><big>Reinforcement learning</big></big></big></big></big>

---

---

<id=tocheading><big><big><big><big>Spis treści</big></big></big></big>
<div id="toc"></div>

---

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [1]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [10]:
output_notebook()
sns.set(font_scale=2.0)

Loading BokehJS ...

Image inclusion
<img src="../mum_figures/" width="100%">

# Uczenie nadzorowane, nienadzorowane, ze wzmocnieniem
1. __poznawcze__ pod _nadzorem_ nauczyciela podającego prawidłowe odpowiedzi (akcje do podjęcia)
  * niepraktyczne w zadaniach interaktywnych
2. __behawioralne__ przez interakcję ze __środowiskiem__: uczenie __reinforcement__
  * agent usiłuje rozwiązać zadanie
  * w nieopisanym środowisku agent musi uczyć się z doświadczenia
  * trudne (lub niemożliwe) zdobycie poprawnych odpowiedzi
3. podejścia do uczenia __ze wzmocnieniem__
  * __klasyczne__ przez serię nagród/kar dla osiągnięcia dobrego zachowania
  * __współczesne__ przez programowanie dynamiczne (__planowanie__)
4. reinforcement jest różne od nienadzorowanego
  * RL stara się maksymalizować __sygnał zwrotu__ (_ang_. __reward__)
  * RL nie stara się szukać struktury problemu (chociaż może to być przydatne)
5. exploration - exploitation tradeoff
  * __exploracja__ przeglądanie przestrzeni rozwiązań w poszukiwaniu nowych
  * __eksploatacja__ maksymalne wykorzystanie już znalezionych
  * niemożliwe jest skupienie się wyłączniena jednym 

# Elementy uczenia ze wzmocnieniem
1. __polityka__ (_ang_. __policy__) definiuje sposób zachowania agenta w danym momencie
  * odwzorowanie ze stanów do akcji do podjęcia w tym stanie
  * tablica, albo proces wyszukiwania
  * zwykle (często) stochastyczne
2. __sygnał wzmocnienia__ (_ang_. __reward__) określa cel
  * w każdym kroku środowisko wysyła __pojedynczy__ sygnał wzmocnienia
  * agent maksymalizuje __całkowity__ reward
  * na podstawie reward agnet moze zmienić swoją politykę
  * agent __nie ma__ wpływu na sam sposób generowania sygnału 
  * reward to (stochastyczna) funkcja stanu i podjętej akcji  
3. funkcja __wartości__ (_ang_. __value__) stanu
  * określa co jest korzystne dla agenta __na dłuższą__ metę
  * wartość stanu, to sumaryczne (wartość oczekiwana) wzmocnienie (reward), które agent może zdobyć w przyszłości __począwszy__ od tego stanu
    * jesli baterie odkurzacza są prawie naładowane, to wartość tego stanu jest wyższa
  * reward stanu może być niskie (zmęczenie organizmu) ale wartość stanu wysoka (już blisko szczytu)
    * i odwrotnie
  * wartość przeliczana przez cały okres działania agenta
4. model środowiska
  * symuluje działanie rzeczywistego środowiska

## Przykłady (Sutton)
1. gra w szachy i wybór ruchu 
  * zależny od sytuacji na planszy
  * prawdopodobnych odpowiedzi na wiele ruchów wprzód
2. kontroler maszyn w czasie rzeczywistym
  * zależny od kosztu i zysków
  * granicznych warunków
3. młody gepard (gazela, robot, ...) wstajacy na nogi zaraz po urodzeniu
4. samojezdny odkurzacz
  * stan baterii
  * stan środowiska
  * częściowa wiedza o środowisku
5. i wiele innych

## Gra w kółko i krzyżyk
1. minimax nie znajdzie optymalnej startegii
  * wybiera akcję, po której, przy __najlepszym__ ruchu przeciwnika, __nie__ przegra
2. programowanie dynamiczne wymaga __pełnego__ opisu środowiska (stanów, akcji, prawdopodobieństw)
3. reinforcement z funkcją wartości stanów
  * wszystkie stany wygrane uzyskują wartość $1$
  * przegrywające $0$
  * wszystkie inne $0.5$ 
    * brak innej wiedzy
  * RL rozpoczyna grę z przeciwnikiem
    * większość ruchów __zachłannych__: do stanu o __wyższej__ wartości; czasem losowe
      * eksploracja -- eksploatacja
  * stany zmieniają wartość
    $$V(s)=V(s)+\alpha\left[V(s')-V(s)\right]$$      * to eksploatacja

## Problem k-jednorękich bandytów
1. k-jednorękich bandytów
  * k maszyn
  * każda ma jakąś __nieznaną__ stopę zwrotu
  * akcja: wybór maszyny
  * reward: ile maszyna zwróci po pociągnięciu wajchy
  * algorytm $\epsilon$-zachłanny
  $$A=\begin{cases}
  \underset{a}{\arg\max}Q(a)&\text{z prawdopodobieństwem}\hskip1em 1-\epsilon\\
  losowa&\text{z prawdopodobieństwem}\hskip1em \epsilon
  \end{cases}$$
    * eksploracja - eksploatacja
  * wyniki będą różne w zależności od wartości $\epsilon$
    * $\epsilon=0$ - zachłanny
      * z początku rośnie szybciej, by szybko wyhamować już bez zmian
      * wybiera nieoptymalne akcje
      * optymalne akcje tylko w ułamku doświadczeń
    * $\epsilon>0$
      * tu algorytm wciąż eksploruje
      * większe $\epsilon$ daje większą eksplorację, jednak w końcu zachowuje się gorzej
      * mniejsze $\epsilon$ uczy się wolniej, ale daje w końcu lepsze wyniki
        * można zmniejszać $\epsilon$
    * zysk $\epsilon$-zachłannego zalezy bardzo od zadania

### Estymacja średniej stopy zwrotu
2. estymacja średniej stopy zwrotu
  $$\begin{align}
  Q_{n+1}&=\frac{1}{n}\sum_{t=1}^nR_t=\frac{1}{n}\left[R_n+\sum_{t=1}^{n-1}R_t\right]\\
  &=\frac{1}{n}\left[R_n+(n-1)Q_n\right]=\frac{1}{n}\left[R_n+nQ_n-Q_n\right]
  =Q_n+\frac{1}{n}\left[R_n-Q_n\right]
  \end{align}$$
  * $\alpha=\frac{1}{n}$ dla środowiska __stacjonarnego__ gdy bandyci nie zmieniają się
3. Estymacja dla $\alpha$
  $$\begin{align}
  Q_{n+1}&=Q_n+\alpha\left[R_n-Q_n\right]=\alpha{}R_n+(1-\alpha)Q_n\\
  &=\alpha{}R_n+(1-\alpha)\left[\alpha{}R_{n-1}+(1-\alpha)Q_{n-1}\right]\\
  &=\dots\\
  &=\alpha{}R_n+(1-\alpha)\alpha{}R_{n-1}+\dots+(1-\alpha)^{n-1}\alpha{}R_{1}+(1-\alpha)^nQ_1\\
  &=(1-\alpha)^nQ_1+\sum_{t=1}^n\alpha(1-\alpha)^{n-t}R_t
  \end{align}$$
  * suma ważona dla $\alpha\in(0,1]$
  * waga $\alpha(1-\alpha)^{n-t}$ zależy od liczby obserwacji wstecz
  * jeśli $\alpha=11$, to $1-\alpha=0$ i wszystkie istotne wagi idą do ostatniej reward $R_n$
    * konwencja: $0^0=1$
  
  * jeśli __parametr kroku__ $\alpha$ będzie __odpowiednio__ zmniejszany, to algorytm zbieżny
  $$\sum_{k=1}^\infty\alpha_n(a)=\infty\hskip1em\text{oraz}\hskip1em\sum_{k=1}^\infty\alpha_n^2(a)<\infty$$
    * niespełnienie drugiego warunku może być przydatne w __niestacjonarnym__ (czy niestabilnym) środowisku
  * to metoda __temporal difference__
4. optymistyczne wartości początkowe
  * standardowo $Q_1=0$
  * optymistyczna wartość większa od rzeczywistej spowoduje większą eksplorację
  * praktyczne dla stacjonarnych, może zaburzyć dla niestacjonarnych

  
### Upper confidence bound
1. Wybór akcje według $$A_t=\underset{a}{\arg\max}\left[Q_t(a)+c\sqrt{\frac{\log\,t}{N_t(a)}}\,\right]$$
gdzie dwa składniki 
  * $Q_t(a)$ to estymacja wartości akcji $a$
  * miara niepewności (wariancji) $\sqrt{\frac{\log\,t}{N_t(a)}}$
2. UCB radzi sobie bardzo dobrze w zadaniach typu k-bandytów  
  * może być słabiej gdy zadania są niestacjonarne
  * czasem problem z radzeniem sobie z całą przestrzenią rozwiązań

### Upper Confidence bound applied to Trees UCT (albo MCTS - Monte Carlo Tree Search)
  * drzewo gry
  * przeciwnicy wybierają ruch na zmianę
  * w każdym cyklu rozwijając drzewo
    1. gracz wybiera optymalny wierzchołek wydług miary UCB
    2. z wybranego węzła (liścia) wykonuje wszystkie możliwe ruchy
    3. dla każdego z nich wykonuje __losowe symulacje__ gier zapamiętując wyniki
    4. poprawia wartości $Q_t(a)$
      * dla liścia uśrednia wyniki wszystkich gier i zapisuje z $N_t(a)=1$
      * poprawia wartości $Q_t(A_t),\;t=t-1,\dots,0$ po drodze aż do korzenia
  * dobrze sprawdza się dla gier
  * przeszukuje w drzewie __tylko__ poddrzewa, które dają nadzieje
  * jeden z pierwszych algorytmów, który dał w miarę sensowne wyniki w Go

### Metody gradientowe (!?!?!?!?!?!!!!?)
1. Podstawowe metody estymują wartości akcji i na tej podstawie wybierają najlepszą
2. można zdefiniować __preferencję__ dla akcji $H_t(a)$
  * preferencja __nie ma__ interpretacji w sensie nagrody (reward)
  * im wyższa preferencja __względem innych__ tym częściej będzie wybrana
  
  $$\pi_t(a)=p(A_t=a)=\frac{\exp(H_t(a))}{\sum_b\exp(H_t(b))}$$
3. Algorytm __stochastycznego wzrostu gradientu__

  $$\begin{align}
  H_{t+1}&=H_{t}+\alpha\frac{\partial\mathbb{E}[R_t]}{\partial{}H_t(a)}\\
  \mathbb{E}[R_t]&=\sum_b\pi_t(b)q_\ast(b)
  \end{align}$$
  gdzie $\mathbb{E}[R+t]$ jest miarą jakości, a $q_\ast(b)$ jest (nieznaną) __optymalną__ funkcją state-value
  * pochodna jakości działania względem __preferencji__
4. jaką postać ma gradient? jak policzyć pochodną oczekiwanej efektywności po preferencji $H_t(a)\dots$?

  $$\begin{align}
\frac{\partial\mathbb{E}[R_t]}{\partial{}H_t(a)}&=\frac{\partial}{\partial{}H_t(a)}\sum_bq_\ast\pi_t(b)=\sum_bq_\ast\frac{\partial\pi_t(b)}{\partial{}H_t(a)}\\
&=\sum_bq_\ast\frac{\partial}{\partial{}H_t(a)}\frac{\exp(H_t()b)}{\sum_c\exp(H_t(c))}\\
&\\
&\hskip3em\text{można odjąć wartość skalarną $X_t$ niezależną od $b$}\\
&\hskip3em\text{bo $\pi_t()$ sumują się do $1$, a gradienty do $0$}\\
&\\
&=\sum_b(q_\ast-X_t)\frac{\partial}{\partial{}H_t(a)}\frac{\exp(H_t(b))}{\sum_c\exp(H_t(c))}\\
&\\
&\hskip3em\text{gradient $\pi_t(b)$, jako funkcji softmax, ma postać}\\
&\hskip5em\frac{\partial\pi_t(b)}{\partial{}H_t(a)}=\pi_t(b)[1_{a=b}-\pi_t(a)]\\
&\\
&=\sum_b(q_\ast-X_t)\,\pi_t(b)\,[1_{a=b}-\pi_t(a)]\\
&=\mathbb{E}_\pi\left[(q_\ast-X_t)[1_{a=b}-\pi_t(a)]\right]\\
&\\
&\hskip3em\text{ponieważ optymalna action-value $\mathbb{E}[R_t\mid A_t]=q_\ast(A_t)$}\\
&\hskip3em\text{niech $q_\ast(A_t)=R_t$ ($R_t$ nieskorelowane)}\\
&\hskip3em\text{oraz przyjąć baseline $X_t=Q_{t+1}$ - średniej wartości zwrotu}\\
&\\
&=\mathbb{E}_\pi\left[(R_t-Q_{t+1})[1_{a=b}-\pi_t(a)]\right]
\end{align}$$

  gdzie $Q_{t+1}=1/t\sum_{k=1}^tR_k$
5. W ten sposób mamy __wzory na poprawę preferencji__
  * dla wybranej akcji $A_t$ $$H_{t+1}(A_t)=H(A_t)+\alpha(R_t-Q_{t+1})(1-\pi_t(A_t))$$
  * dla akcji niewybranych $a\neq A_t$ $$H_{t+1}(a)=H(A_t)-\alpha(R_t-Q_{t+1})\pi_t(a)$$
  * składnik $Q_{t+1}$ służy jako __odniesienie__ z którym porównywana jest wartość $R_t$ uzyskana
    * jeśli $R_t$ jest __większa__, to preferencja będzie __zwiększana__
    * preferencje __nie wybranych__ akcji będą zmniejszane __proporcjonalnie__ do szans ich wybrania $\pi_t(a)$
  * ominięcie składnika odniesienia wpływa osłabiająco na algorytm

## Meęèéêëėētoóôöòõœøōdy (niektóre)
(bez określonego porządku)
1. metody ewolucyjne
  * symulują zachowanie agentów w czasie całego ich życia
  * nie określają wartości stanów
  * nie kontaktują się ze środowiskiem
  * abstrahują od definicji policy, której szukają, jako funkcji stanów w akcje
2. przeszukiwanie przestrzeni rozwiązań
  * optymalizacja policy jako zbioru numerycznych parametrów i poszukiwanie kierunku poprawy
  * __policy gradient__
3. przeszukiwanie metodami Monte-Carlo
  * określenie lepszych/gorszych obszarów
  * optymalizacja dylematu exploration-exploitation
  * __upper confidence bound__
  * także proste metody losowe i optymalizacji __hill climbing__
4. __programowanie dynamiczne__

## Law of Effect
_Of several responses made to the same situation, those which are accompanied or closely followed by satisfaction to the animal will, other things being equal, be more firmly connected with the situation, so that, when it recurs, they will be more likely to recur; those which are accompanied or closely followed by discomfort to the animal will, other things being equal, have their connections with this situation weakened, so that, when it recurs, they will be less likely to occur. The greater the satisfaction or discomfort, the greater the strengthening or weakening of the bond_ (Edward Thorndike, 1911)

# Credit assignment problem
1. __programowanie dynamiczne__
  * decyzje podejmowane w krokach
  * żądanie niskiego __aktualnego__ kosztu musi być zrównoważone z __wysokim__ kosztem w przyszłości
  * kredyt musi być zrównoważony przez wszystkie podejmowane decyzje
  
<big>___W jaki sposób agent może podnieść swoje długo-terminowe wyniki w stochastycznym środowisku gdy osiągnięcie zysków może się wiązać kosztem wyników krótko-terminowych?___</big>



# Procesy decyzyjne Markowa
1. agent działa zgodnie z założeniami __skończenie stanowego procesu Markowa__
  <img style="float: right;" src="../mum_figures/markov_process.png" width="45%">
  * proces rozwija się __probabilistycznie__
  * ma __skończoną__ liczbę stanów
  * __nie zawiera__ statystyk poprzedniego zachowania
  * dla każdego stanu istnieje __skończona__ liczba akcji możliwych do podjęcia
  * każda akcja jest związana z pewnym __kosztem__
  * wszystko dzieje się w czasie __dyskretnym__
  
    __Stan środowiska jest pewnym _podsumowaniem_ całego dotychczasowego doświadczenia agenta z tym środowiskiem. Informacja konieczna dla agenta dla przewidywania przyszłości jest zawarta w tym podsumowaniu__

## Akcje
2. dla każdego stanu $s$ istnieje __skończony__ zbiór akcji $a_{s,s'}$
  <img style="float: right;" src="../mum_figures/markov_process.png" width="45%">
  * akcja przeprowadza __probabilistycznie__ stan $S_t=s$ w stan $S_{n+1}=s'$  
    1. $p_{s,s'}(a)\ge0$
    2. $\sum_{s'}p_{s,s'}(a)=1$
  ---
<big>__Własność Markowa: prawdopodobieństwo przejścia ze stanu $s$ w stan $s'$ zależy _wyłącznie_ od aktualnego stanu $s$ oraz akcji $a_{s,s'}$__</big>
  

## Reward
3. każde przejście jest związane z reward (kosztem) $r(s, a, s')$ dla agenta
<img style="float: right;" src="../mum_figures/markov_process.png" width="45%">  
  * $r(s, a_{s,s'}, s')$ jest ustaloną funkcją
  * szachy: +1 za wygraną, -1 za przegraną, 0 w innym przypadku
  * odkurzacz: 
    * dodatnie wartości za odkurzanie dawno nie odwiedzanego terenu
    * dodatnie ale mniejsze za odkurzanie terenu widzanego niedawno
    * zerowa za utrzymywanie baterii w stanie naładowanym
    * wysoka ujemna za rozładowanie baterii
  * reward jest __deklaracją celu__
    * __nie jest__ deklaracją sposobu jego osiągnięcia

## Zwrot (return)
  <big>__Celem i zadaniem procesu jest maksymalizacja _oczekiwanej wartości skumulowanej sumy_ uzyskanych wartości skalarnych (reward)__</big>
1. Celem jest maksymalizacja oczekiwanej wartości skumulowanych reward
  * najprościej $$G_t=R_{t+1}+R_{t+2}+\dots+R_{T}$$
  gdzie $T$ jest numerem ostatniego kroku
2. __epizod__ oznacza naturalną skończoną sekwencję zdarzeń agent-środowisko
  * pojedyncze gry, codzienne odkurzanie, etc.
  * epizod ma stan końcowy (terminalny)
    * po końcu następuje __powrót__ do ustalonego (lub określanego inaczej, np. losowanego) stanu początkowego
  * zadania w epizodach to, nomen omen, __zdania epizodyczne__
3. zadania __ciągłe__
  * nie dzielą się na epizody
  * wtedy $$G_t=R_{t+1}+\gamma{}R_{t+2}+\gamma{}R_{t+3}\dots=\sum_{k=0}^\infty\gamma^kR_{t+k+1}$$
  * $0\ge\gamma\ge1$ jest czynnikiem __discount__
  * regulacja $\gamma$ pozwala sterować na ile agent interesuje się przyszłością
    * małe gamma to zainteresowanie celami bliskimi
    * $\gamma=0$ daje agenta __krótkowzrocznego__ (ang. myopic) zainteresowanego __tylko__ chwilą obecną
    * dla $\gamma$ bliższego $1$ ważniejsza staje się przyszłość (cała historia)
4. Zunifikowana notacja to $$G_t=\sum_{k=0}^{T-t-1}\gamma^kR_{t+k+1}$$ 
  * w tym $T=\infty$
  * a także $\gamma=1$ (ale __nie jednocześnie z $T=\infty$)


## Policy (strategia, polityka)
4. __policy__ $\pi$ to __odwzorowanie stanów w akcje__
  * deterministyczne $\pi(s)$ opisuje akcję podejmowaną w stanie s
  * stochastyczne $\pi_t(a\mid s)=P(A_t=a\mid S_t=s)$
  * policy jest __dopuszczalna__ (ang. admissible) jeśli $\pi(s)\in\mathcal{A}_s$ gdzie $\mathcal{A}_s$ jest zbiorem wszystkich możliwych akcji w stanie $s$
  * policy może być
    * __niestacjonarna__ jeśli zmienia się w czasie
    * __stacjonarna__ jeśli jest niezależna od kroku
5. Proces Markowa może być
  * __stacjonarny__, gdy $p_{s,s'}$ nie zmieniają się
  * __niestacjonarny__ w przeciwnym wypadku

## Funkcje wartości (value)
1. funkcja __wartości__ (_ang_. __value__, __state-value__) $v_\pi(s)$ określa jakość stanu $s$ przy polityce (policy) $\pi$
  * czyli oczekiwana wartość zwrotu w stanie $s$

  $$\begin{align}
v_\pi(s)&=\mathbb{E}_\pi\left[G_t\mid S_t=s\right]\\
&=\mathbb{E}_\pi\left[\sum_{k=0}^\infty\gamma^kR_{t+k+1}\mid S_t=s\right]
\end{align}$$
2. podobnie wartość podjęcia akcji (_ang_. __action-value__) $a$ w stanie $s$ w polityce $\pi$ 

  $$\begin{align}
q_\pi(s, a)&=\mathbb{E}_\pi\left[G_t\mid S_t=s, A_t=a\right]\\
&=\mathbb{E}_\pi\left[\sum_{k=0}^\infty\gamma^kR_{t+k+1}\mid S_t=s, A_t=a\right]
\end{align}$$

## Równanie Bellmana
3. wartości stanu użyte w trakcie uczenia spełniają zależności rekurencyjne

  $$\begin{align}
v_\pi(s)&=\mathbb{E}_\pi\left[G_t\mid S_t=s\right]\\
&=\mathbb{E}_\pi\left[\sum_{k=0}^\infty\gamma^kR_{t+k+1}\mid S_t=s\right]\\
&=\mathbb{E}_\pi\left[R_{t+1}+\gamma\sum_{k=0}^\infty\gamma^kR_{t+k+2}\mid S_t=s\right]\\
\end{align}$$

  * ta wartość oczekiwana zależy od __kolejnego stanu $s'$__, a więc od podjętej w stanie $s$ akcji $a$ i rewardów z tym związanych
$$\begin{align}
v_\pi(s)&=\mathbb{E}_\pi\left[R_{t+1}+\gamma\sum_{k=0}^\infty\gamma^kR_{t+k+2}\mid S_t=s\right]\\
&=\sum_a\pi(a\mid s)\sum_{s',r}p(s',r\mid s,a)\left[r+\gamma\left[\sum_{k=0}^\infty\gamma^kR_{t+k+2}\mid S_{t+1}=s'\right]\right]\\
&=\sum_a\pi(a\mid s)\sum_{s',r}p(s',r\mid s,a)\left[r+\gamma{}v_\pi(s')\right]
\end{align}$$
  * to jest __równanie Bellmana__
    * spojrzenie __wprzód__ z aktualnego stanu do __możliwych__ następników
    * agent wybiera jedną z możliwych akcji $a$
    * __środowisko__ odpowiada jednym z możliwych stanów $s'$ oraz rewardem $r$
  * równanie Bellmana waży prawdopodobieństwa wystąpienia stanowiąc __kryterium optymalności__
  * $v_\pi$ jest jedynym mozliwym rozwiązaniem
  * większość algorytmów RL będzie się starało rozwiązać równanie w jakiś sposób

## Kryterium optymalności Bellmana
1. Równanie Bellmana stanowi __kryterium optymalności__ dla state-value i action-value funkcji
  * state value
  $$\begin{align}
v_\ast(s)&=\underset{a}{\max}q_{\pi_\ast}(s, a)\\
&=\underset{a}{\max}\sum_{s',r}p(s',r\mid s,a)\left[r+\gamma{}v_\pi(s')\right]
\end{align}$$

  <img style="float: center;" src="../mum_figures/bellman_backup.png" width="75%">

  * action value
  $$\begin{align}
q_\ast(s,a)&=\mathbb{E}\left[R_{t+1}+\gamma\underset{a'}{\max}q_\ast(S_{t+1},a')\mid S_t=s, A_t=a\right]\\
&=\sum_{s',r}p(s',r\mid s,a)\left[r+\gamma\underset{a'}{\max}q_\ast(s',a')\right]
\end{align}$$

2. możliwa jest __ewaluacja policy__ przez iteracyjne dochodzenie do rozwiązania optymalnego
  * action-value to __policy improvement__
    * powolna operacja wymagająca w cyklach
      * ewaluacji wszystkich akcji
      * poprawy 
  * __value iteration__ jest prostsze
  $$\begin{align}
v_{k+1}(s)&=\underset{a}{\max}\mathbb{E}\left[R_{t+1}+\gamma{}v_k(S_{t+1})\mid S_t=s, A_t=a\right]\\
&=\underset{a}{\max}\sum_{s',r}p(s',r\mid s,a)\left[r+\gamma{}v_k(s')\right]
\end{align}$$
    * odpowiada to przekształceniu równania Bellmana w regułę poprawy
    
  <img style="float: center;" src="../mum_figures/bellman_backup_optimal.png" width="75%">

